In [1]:
pip install deeplake

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import deeplake

In [3]:
ds = deeplake.load('hub://activeloop/fer2013-train')
trainx2,trainy2=ds.images,ds.labels
trainx2=np.array(trainx2)
trainy2=np.array(trainy2)
ds = deeplake.load('hub://activeloop/fer2013-public-test')
testx2,testy2=ds.images,ds.labels
testx2=np.array(testx2)
testy2=np.array(testy2)

\

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-train



\

...

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-public-test



\

hub://activeloop/fer2013-public-test loaded successfully.



In [4]:
trainx2.shape

(28709, 48, 48)

In [5]:
trainx2=np.dstack([trainx2] * 3)
testx2=np.dstack([testx2]*3)
trainx2 = trainx2.reshape(-1, 48,48,3)
testx2= testx2.reshape (-1,48,48,3)
trainx2.shape

(28709, 48, 48, 3)

In [11]:
from keras.preprocessing.image import img_to_array, array_to_img

trainx2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in trainx2])
testx2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in testx2])

In [12]:
from tensorflow.keras.applications import VGG16
input_layer=layers.Input(shape=(64,64,3))
model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)

model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [13]:
last_layer=model_vgg16.output
flatten=layers.Flatten()(last_layer)
dense1=layers.Dense(4096,activation='relu')(flatten)
dense1=layers.Dense(4096,activation='relu')(flatten)
output_layer=layers.Dense(7,activation='softmax')(flatten)
model=models.Model(inputs=input_layer,outputs=output_layer)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [14]:
for layer in model.layers[:-1]:
    layer.trainable=False
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(trainx2, trainy2, epochs=5,
                    validation_data=(testx2, testy2))

Epoch 1/5
898/898 [==============================] - 19s 20ms/step - loss: 7.7752 - accuracy: 0.2795 - val_loss: 6.2888 - val_accuracy: 0.2895
Epoch 2/5
898/898 [==============================] - 16s 18ms/step - loss: 4.9602 - accuracy: 0.3198 - val_loss: 6.0642 - val_accuracy: 0.2742
Epoch 3/5
898/898 [==============================] - 16s 18ms/step - loss: 4.4175 - accuracy: 0.3370 - val_loss: 6.3755 - val_accuracy: 0.2784
Epoch 4/5
898/898 [==============================] - 17s 19ms/step - loss: 4.3156 - accuracy: 0.3424 - val_loss: 4.6162 - val_accuracy: 0.3369
Epoch 5/5
898/898 [==============================] - 18s 20ms/step - loss: 4.2089 - accuracy: 0.3474 - val_loss: 4.9384 - val_accuracy: 0.3413


In [16]:
test_loss, test_acc = model.evaluate(testx2,  testy2, verbose=2)
test_acc

113/113 - 2s - loss: 4.9384 - accuracy: 0.3413 - 2s/epoch - 15ms/step


0.3413206934928894

In [18]:
model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 128)       147584    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 128)       0         
 g2D)                                                 

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(trainx2, trainy2, epochs=5,
                    validation_data=(testx2, testy2))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


898/898 [==============================] - 62s 60ms/step - loss: 3.5154 - accuracy: 0.2506 - val_loss: 1.8132 - val_accuracy: 0.2494
Epoch 2/5
898/898 [==============================] - 53s 59ms/step - loss: 1.8114 - accuracy: 0.2513 - val_loss: 1.8115 - val_accuracy: 0.2494
Epoch 3/5
898/898 [==============================] - 53s 59ms/step - loss: 1.8108 - accuracy: 0.2513 - val_loss: 1.8111 - val_accuracy: 0.2494
Epoch 4/5
898/898 [==============================] - 54s 60ms/step - loss: 1.8108 - accuracy: 0.2513 - val_loss: 1.8120 - val_accuracy: 0.2494
Epoch 5/5
898/898 [==============================] - 54s 60ms/step - loss: 1.8106 - accuracy: 0.2513 - val_loss: 1.8119 - val_accuracy: 0.2494
